In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_250.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_201.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_269.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_126.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_010.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_199.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_139.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_151.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_281.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_103.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_211.jpg
/kaggle/input/betel-leaf-dataset/On Field/Diseased Leaf/OF_Diseased_Leaf_002.jpg
/kaggle/input/betel-leaf-dat

In [11]:
# Standard imports
import os
import json
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import imagehash

In [18]:
import os, json
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image

# Configuration
DATA_DIR = "/kaggle/input/betel-leaf-dataset/Controlled Environment"
OUT_DIR = '/kaggle/working/eda_outputs'
os.makedirs(OUT_DIR, exist_ok=True)
CHECKPOINT = Path(OUT_DIR) / 'checkpoint.json'

# Define expected class folders (exact names from dataset)
CLASS_NAMES = ["Healthy", "Diseased", "Dried"]

# Utility: read/write checkpoint
def load_checkpoint():
    if CHECKPOINT.exists():
        return json.loads(CHECKPOINT.read_text())
    return {}

def save_checkpoint(d):
    CHECKPOINT.write_text(json.dumps(d, indent=2))

cp = load_checkpoint()

# Wrapper to compute or load cached numpy/pandas files
def cached_path(name, ext='json'):
    return Path(OUT_DIR) / f'{name}.{ext}'

# %%
# Step 0: discover image file paths
index_file = cached_path('image_index', 'csv')

# যদি ফাইল খালি থাকে → ডিলিট করে দাও
if index_file.exists() and os.path.getsize(index_file) == 0:
    print("Found empty index file, deleting...")
    index_file.unlink()
    cp['image_index'] = False
    save_checkpoint(cp)

if (not cp.get('image_index')) or (not index_file.exists()):
    print('Discovering image files...')
    records = []
    for cls in CLASS_NAMES:
        cls_dir = Path(DATA_DIR) / cls
        if not cls_dir.exists():
            print(f'Warning: expected class folder {cls_dir} not found. Check DATA_DIR and CLASS_NAMES.')
            continue
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            for p in cls_dir.rglob(ext):
                records.append({'path': str(p), 'class': cls})

    df_index = pd.DataFrame(records)
    if len(df_index) > 0:
        df_index.to_csv(index_file, index=False)
        cp['image_index'] = True
        save_checkpoint(cp)
    else:
        print("No images found in dataset folders!")
        df_index = pd.DataFrame(columns=['path', 'class'])
else:
    print('Loading cached image index...')
    df_index = pd.read_csv(index_file)

print(f"Total images found: {len(df_index)}")

# %%
# Helper functions for image loading and safe conversion
def pil_load(p, as_rgb=True):
    img = Image.open(p).convert('RGB') if as_rgb else Image.open(p)
    return np.array(img)


Loading cached image index...


EmptyDataError: No columns to parse from file

In [17]:
from pathlib import Path

DATA_DIR = Path("/kaggle/input/betel-leaf-dataset/Controlled Environment")

print("Listing subfolders inside Controlled Environment:")
for p in DATA_DIR.iterdir():
    if p.is_dir():
        print("DIR :", p.name)
    else:
        print("FILE:", p.name)


Listing subfolders inside Controlled Environment:
DIR : Diseased
DIR : Dried
DIR : Healthy


In [20]:
from pathlib import Path
import os

index_file = Path("/kaggle/working/eda_outputs/image_index.csv")

# যদি পুরোনো খালি ফাইল থাকে → ডিলিট করো
if index_file.exists():
    print("Deleting old index file...")
    index_file.unlink()


Deleting old index file...


In [21]:
import os, json
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image

# Configuration
DATA_DIR = "/kaggle/input/betel-leaf-dataset/Controlled Environment"
OUT_DIR = '/kaggle/working/eda_outputs'
os.makedirs(OUT_DIR, exist_ok=True)
CHECKPOINT = Path(OUT_DIR) / 'checkpoint.json'

# Define expected class folders (exact names from dataset)
CLASS_NAMES = ["Healthy", "Diseased", "Dried"]

# Utility: read/write checkpoint
def load_checkpoint():
    if CHECKPOINT.exists():
        return json.loads(CHECKPOINT.read_text())
    return {}

def save_checkpoint(d):
    CHECKPOINT.write_text(json.dumps(d, indent=2))

cp = load_checkpoint()

# Wrapper to compute or load cached numpy/pandas files
def cached_path(name, ext='json'):
    return Path(OUT_DIR) / f'{name}.{ext}'

# %%
# Step 0: discover image file paths
index_file = cached_path('image_index', 'csv')

# যদি ফাইল খালি থাকে → ডিলিট করে দাও
if index_file.exists() and os.path.getsize(index_file) == 0:
    print("Found empty index file, deleting...")
    index_file.unlink()
    cp['image_index'] = False
    save_checkpoint(cp)

if (not cp.get('image_index')) or (not index_file.exists()):
    print('Discovering image files...')
    records = []
    for cls in CLASS_NAMES:
        cls_dir = Path(DATA_DIR) / cls
        if not cls_dir.exists():
            print(f'Warning: expected class folder {cls_dir} not found. Check DATA_DIR and CLASS_NAMES.')
            continue
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            for p in cls_dir.rglob(ext):
                records.append({'path': str(p), 'class': cls})

    df_index = pd.DataFrame(records)
    if len(df_index) > 0:
        df_index.to_csv(index_file, index=False)
        cp['image_index'] = True
        save_checkpoint(cp)
    else:
        print("No images found in dataset folders!")
        df_index = pd.DataFrame(columns=['path', 'class'])
else:
    print('Loading cached image index...')
    df_index = pd.read_csv(index_file)

print(f"Total images found: {len(df_index)}")

# %%
# Helper functions for image loading and safe conversion
def pil_load(p, as_rgb=True):
    img = Image.open(p).convert('RGB') if as_rgb else Image.open(p)
    return np.array(img)


Discovering image files...
Total images found: 893


In [24]:
# Step 1: Compute per-image stats (mean, std, brightness, saturation)

# class কলামের নাম পরিবর্তন
df_index = df_index.rename(columns={'class': 'label'})

if not cp.get('per_image_stats'):
    print('Computing per-image statistics...')
    stats = []
    for row in tqdm(df_index.itertuples(), total=len(df_index)):
        p = row.path
        try:
            img = pil_load(p)
            h, w, _ = img.shape
            mean_rgb = img.reshape(-1, 3).mean(axis=0).tolist()
            std_rgb = img.reshape(-1, 3).std(axis=0).tolist()
            # brightness: use V from HSV
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            v_mean = float(hsv[:, :, 2].mean())
            s_mean = float(hsv[:, :, 1].mean())
            # sharpness via laplacian variance
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            lap_var = float(cv2.Laplacian(gray, cv2.CV_64F).var())
            stats.append({
                'path': p,
                'label': row.label,   # এখন row.class এর বদলে row.label
                'height': h, 'width': w,
                'mean_r': mean_rgb[0], 'mean_g': mean_rgb[1], 'mean_b': mean_rgb[2],
                'std_r': std_rgb[0], 'std_g': std_rgb[1], 'std_b': std_rgb[2],
                'v_mean': v_mean, 's_mean': s_mean, 'lap_var': lap_var
            })
        except Exception as e:
            print('Error reading', p, e)

    df_stats = pd.DataFrame(stats)
    df_stats.to_csv(cached_path('per_image_stats', 'csv'), index=False)
    cp['per_image_stats'] = True
    save_checkpoint(cp)
else:
    print('Loading cached per-image stats...')
    df_stats = pd.read_csv(cached_path('per_image_stats', 'csv'))


Computing per-image statistics...


100%|██████████| 893/893 [57:52<00:00,  3.89s/it]


In [27]:
# Step 2: Per-class summaries and histogram plots
if not cp.get('plots'):
    print('Generating plots...')

    # Per-class mean/std table
    class_summ = df_stats.groupby('label').agg({
        'mean_r':'mean','mean_g':'mean','mean_b':'mean',
        'std_r':'mean','std_g':'mean','std_b':'mean',
        'v_mean':'mean','s_mean':'mean','lap_var':'mean',
        'path':'count', 'width':'mean', 'height':'mean'
    }).rename(columns={'path':'count'})
    class_summ.to_csv(cached_path('class_summary', 'csv'))

    # RGB histograms (global and per class)
    bins = 256
    all_r = df_stats['mean_r'].values
    all_g = df_stats['mean_g'].values
    all_b = df_stats['mean_b'].values
    plt.figure(figsize=(8,5))
    plt.hist(all_r, bins=50, alpha=0.6, label='R')
    plt.hist(all_g, bins=50, alpha=0.6, label='G')
    plt.hist(all_b, bins=50, alpha=0.6, label='B')
    plt.legend(); plt.title('Per-image mean RGB distribution')
    plt.savefig(Path(OUT_DIR)/'hist_mean_rgb.png'); plt.close()

    # HSV brightness distribution
    plt.figure(figsize=(8,5))
    plt.hist(df_stats['v_mean'], bins=50)
    plt.title('Per-image mean V (brightness) distribution')
    plt.savefig(Path(OUT_DIR)/'hist_brightness.png'); plt.close()

    # Aspect ratio scatter
    df_stats['aspect'] = df_stats['width'] / df_stats['height']
    plt.figure(figsize=(6,6))
    plt.scatter(df_stats['width'], df_stats['height'], s=6)
    plt.xlabel('width'); plt.ylabel('height'); plt.title('Resolution distribution')
    plt.savefig(Path(OUT_DIR)/'resolution_scatter.png'); plt.close()

    # Sharpness (laplacian) per class boxplot
    plt.figure(figsize=(8,5))
    df_stats.boxplot(column='lap_var', by='label')
    plt.title('Laplacian variance (sharpness) per class'); plt.suptitle('')
    plt.savefig(Path(OUT_DIR)/'sharpness_box.png'); plt.close()

    cp['plots'] = True
    save_checkpoint(cp)

else:
    print('Plots already generated (cached).')


Plots already generated (cached).


In [29]:
# Step 3: Perceptual hashing to detect duplicates / near-duplicates
from collections import defaultdict
import imagehash

if not cp.get('duplicates'):
    print('Computing perceptual hashes...')
    hashes = defaultdict(list)
    for row in tqdm(df_index.itertuples(), total=len(df_index)):
        try:
            ph = str(imagehash.phash(Image.open(row.path)))
            hashes[ph].append(row.path)
        except Exception as e:
            print('hash error', row.path, e)

    # find groups with more than one entry
    dup_groups = [v for v in hashes.values() if len(v) > 1]
    # save a small sample: first 200 groups
    dup_sample = dup_groups[:200]
    with open(cached_path('duplicate_groups', 'json'), 'w') as f:
        json.dump(dup_sample, f, indent=2)
    cp['duplicates'] = True
    save_checkpoint(cp)

else:
    print('Loading cached duplicate groups...')
    with open(cached_path('duplicate_groups', 'json')) as f:
        dup_sample = json.load(f)

print(f'Found {len(dup_sample)} duplicate hash groups (sample saved).')


Computing perceptual hashes...


100%|██████████| 893/893 [05:52<00:00,  2.53it/s]

Found 3 duplicate hash groups (sample saved).


In [31]:
# Step 4: Augmentation probe - make small grid of augmented examples per class
if not cp.get('augmentation_probe'):
    print('Running small augmentation probe...')

    # Use simple OpenCV/PIL transforms so no external libs required
    def augment_examples(img_arr):
        outs = []
        # original
        outs.append(img_arr)
        # horizontal flip
        outs.append(np.fliplr(img_arr))
        # rotate 15 deg
        h, w = img_arr.shape[:2]
        M = cv2.getRotationMatrix2D((w/2, h/2), 15, 1)
        outs.append(cv2.warpAffine(img_arr, M, (w, h)))
        # random crop center 80%
        ch, cw = int(h*0.8), int(w*0.8)
        sh, sw = (h-ch)//2, (w-cw)//2
        outs.append(img_arr[sh:sh+ch, sw:sw+cw])
        # blur
        outs.append(cv2.GaussianBlur(img_arr, (5,5), 0))
        # color jitter approximation: add/sub small value to V in HSV
        hsv = cv2.cvtColor(img_arr, cv2.COLOR_RGB2HSV).astype(np.int16)
        hsv[:,:,2] = np.clip(hsv[:,:,2] + 15, 0, 255)
        outs.append(cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB))
        return outs

    probe_dir = Path(OUT_DIR) / 'augmentation_probe'
    probe_dir.mkdir(exist_ok=True)

    # ⚠️ আগের ধাপে আমরা 'class' কলাম rename করে 'label' করেছি
    for cls in CLASS_NAMES:
        sample_paths = df_index[df_index['label'] == cls] \
            .sample(min(5, df_index[df_index['label'] == cls].shape[0]), random_state=1)['path'].tolist()
        for i, p in enumerate(sample_paths):
            img = pil_load(p)
            aug = augment_examples(img)
            # save grid
            rows = []
            for a in aug:
                rows.append(a)
            # create horizontal strip
            strip = np.hstack([cv2.resize(a, (224,224)) for a in rows])
            cv2.imwrite(str(probe_dir / f'{cls}_{i}_probe.jpg'),
                        cv2.cvtColor(strip, cv2.COLOR_RGB2BGR))

    cp['augmentation_probe'] = True
    save_checkpoint(cp)

else:
    print('Augmentation probe already created.')


Running small augmentation probe...


In [33]:
# Step 5: White-balance gray-world check (per-class) and save report
if not cp.get('gray_world'):
    print('Running gray-world white-balance sanity check...')
    df_stats['rg_diff'] = df_stats['mean_r'] - df_stats['mean_g']
    df_stats['rb_diff'] = df_stats['mean_r'] - df_stats['mean_b']
    
    # group by label (not 'class')
    gw = df_stats.groupby('label')[['rg_diff','rb_diff']].mean()
    gw.to_csv(cached_path('gray_world_class_report', 'csv'))
    
    cp['gray_world'] = True
    save_checkpoint(cp)
else:
    print('Gray-world check cached.')


Running gray-world white-balance sanity check...


In [36]:
# Step 6: Class balance and leakage safeguards note
if not cp.get('class_balance'):
    print('Saving class balance summary...')

    # class → label
    class_counts = df_index['label'].value_counts().to_dict()
    with open(cached_path('class_balance', 'json'), 'w') as f:
        json.dump(class_counts, f, indent=2)

    # Leakage: try to find possible shared prefixes in filenames indicating same sample
    # e.g., filenames like sample123_view1.jpg -> group by sample id prefix
    def sample_id_from_path(p):
        name = Path(p).stem
        # crude heuristic: split by '_' and take first token
        return name.split('_')[0]

    df_index['sample_id'] = df_index['path'].apply(sample_id_from_path)
    groups = (
        df_index.groupby(['sample_id'])['path']
        .count()
        .reset_index()
        .rename(columns={'path': 'count'})
    )
    groups.to_csv(cached_path('sample_groups', 'csv'), index=False)

    cp['class_balance'] = True
    save_checkpoint(cp)

else:
    print('Class balance cached.')


Class balance cached.


In [37]:
# Final: summarize everything into a short JSON report
if not cp.get('final_report'):
    print('Writing final JSON report summary...')
    report = {
        'total_images': len(df_index),
        'class_counts': json.loads((Path(OUT_DIR) / 'class_balance.json').read_text()),
        'duplicate_groups_sample': len(dup_sample),
    }
    with open(cached_path('final_report', 'json'), 'w') as f:
        json.dump(report, f, indent=2)
    cp['final_report'] = True
    save_checkpoint(cp)
else:
    print('Final report already exists.')

print('EDA run complete. Outputs saved to', OUT_DIR)
print('Files you can check:')
for p in sorted(os.listdir(OUT_DIR))[:50]:
    print('-', p)


Writing final JSON report summary...
EDA run complete. Outputs saved to /kaggle/working/eda_outputs
Files you can check:
- augmentation_probe
- checkpoint.json
- class_balance.json
- class_summary.csv
- duplicate_groups.json
- final_report.json
- gray_world_class_report.csv
- hist_brightness.png
- hist_mean_rgb.png
- image_index.csv
- per_image_stats.csv
- resolution_scatter.png
- sample_groups.csv
- sharpness_box.png


In [38]:
import json
from pathlib import Path

# ফাইলের path
json_path = Path(OUT_DIR) / "final_report.json"

# লোড করা
with open(json_path, "r") as f:
    data = json.load(f)

# সাধারণ প্রিন্ট
print(data)

# সুন্দরভাবে (pretty print) প্রিন্ট
print(json.dumps(data, indent=4))


{'total_images': 893, 'class_counts': {'Dried': 340, 'Healthy': 333, 'Diseased': 220}, 'duplicate_groups_sample': 3}
{
    "total_images": 893,
    "class_counts": {
        "Dried": 340,
        "Healthy": 333,
        "Diseased": 220
    },
    "duplicate_groups_sample": 3
}
